# 466 Count The Repetition

先定义了一种字符串的表示方式：[str, int] 元组表示 <int> 个 str 串连续出现组合成的新字符串。
    
然后用 [s1, n1] 和 [s2, n2] 分别构造出 S1 和 S2 两个大、长字符串。
    
最后问你：从 S1 中不改变原有顺序地删除任意多个字符，能得到 [S2, M] 中 M 的最大值是？

要得到最终的 M，我们就需要从 S1（那个重复节组成的字符串）里检测出符合 S2 的循环节，即，需要从多少长度里才能修剪出一个 S2。

实际上，[s, n] 这种表示方式非常分散注意力。重要的是那些组成的字符。

我们把 s1 和 s2（循环节）都想象成无限循环的字符串。令 i1 和 i2 是这两个「无限字符串」的索引（他们对 len1 和 len2 取模的结果才对应到 s1、s2 的索引）。无限延长之后，n1 和 n2 对我们来说就不是限制了。但是在这里，鉴于我们要从 s1 * n1 中删除 char 来得到结果，我们会将 i1 的上限定为 len(s1) * n1。超过这个值的 i1 索引没有意义。

现在事情就简单了：这变成了一个「删除元素」问题。当然也别真的去操作字符串，删元素（Python 里做这种事情是很难受的，对吧）。我们只需要对每一个 s2 中的 char，按顺序递增 i1，遇到相等的 char 就增加两根指针；两个 i1、i2 都取模之后再对 s1、s2 索引。在这时我们暂且不考虑 s2 重复的次数。

到最后，我们应该会到一个终点：要么是 i1 达到了上限（也就是 len(s1) * n1 用完了），要么是我们发现了一个循环节（在此处，s1 跟 s2 都刚好归零，非常巧）。在发现了循环节的时候，我们可以直接将此时的 i1 翻整数倍，减少无用计算。注意此时的 i1 并不一定能完全填充 len(s1) * n1，也就是还有需要特别处理的「小尾巴」。（这一步仅仅是为了优化，如果不进行「快速翻倍处理」也不会错。）

最后，我们应该会得到一个需要再次处理的数字，那就是 i2：他指明了在这个「抽象无限循环字符串 s2」最终能达到的索引。但是我们要求的结果是用 S2（s2 * n2）从中向下取整数。这件事情，已经和 s1/n1 无关了。只要简单地做个出发就好了。

Great thanks to @yuanzhi247012 and @i-i。唯一一个我能看得懂的答案。（当然也是因为现在脑子不清醒。睡了。）

In [ ]:
class Solution:
    def getMaxRepetitions(self, s1, n1, s2, n2):
        d, l1, l2, i1, i2 = {}, len(s1), len(s2), 0, 0
        tot = l1 * n1

        while i1 < tot:
            if s1[i1 % l1] == s2[i2 % l2]:
                if (i1 % l1, i2 % l2) in d:
                    prev1, prev2 = d[(i1 % l1, i2 % l2)]
                    cir1, cir2 = i1 - prev1, i2 - prev2
                    count_cir1 = (tot - i1) // cir1
                    i1 += count_cir1 * cir1
                    i2 += count_cir1 * cir2
                    if i1 >= tot: break
                else:
                    d[(i1 % l1, i2 % l2)] = (i1, i2)
                i2 += 1
            i1 += 1
        return i2 // l2 // n2

---------------------------- deprecated --------------------------------------


本质上，是个求删节残存字符串中重复节最多出现的个数的题。但是得考虑到，特殊地，本题中的字符串都是 Duplicated 出来的反复字符串。
如果不利用这个性质，大量的数据就完全无法处理。

我们得从循环节出手。

```
repeatCount = [0] * (n1 + 1)
nextIndex = [0] * (n1 + 1)
j, count = 0, 0

# ...
        for k in range(1, n1 + 1):
            for i in range(len(s1)):
                if s1[i] == s2[j]:
                    j += 1
                    if j == len(s2):
                        j = 0
                        count += 1
            repeatCount[k] = count
            nextIndex[k] = j
```

这段代码能看明白吗？因为我们要从 n1 个 s1 中删除字符得到 k * n2 个 s2，所以我们得遍历 s1，并且记录下来「重合」的那些字符。将它们记录到 count 里面。但是因为我们处理的对象是 [str, int] 这类特殊的定义字符串，因此我们采用 repeatCount 和 nextIndex 数组来记录当前行走到的索引位置，而不去真的生成 s1 * n1 这么大的 str 对象。至于 s2，我们默认其重复节无限（假如最后发现不够用，即 S2 太大填不进去，直接令 M = 0 了事）

然后，在 k in range(i, n1 + 1) 循环里，我们还要做：

```
            for start in range(k):
                if nextIndex[start] == j:
                    prefixCount = repeatCount[start]
                    patternCount = (n1 - start) // (k - start) * (repeatCount[k] - prefixCount)
                    suffixCount = repeatCount[start + (n1 - start) % (k - start)] - prefixCount
                    return (prefixCount + patternCount + suffixCount) // n2
```

k 是我们目前拿到的 s1 循环次数。假如我们遇到 nextIndex[start] == j，也就是已经完成了一次完整的遍历，

In [ ]:
class Solution2:
    def getMaxRepetitions(self, s1: str, n1: int, s2: str, n2: int) -> int:

        repeatCount = [0] * (n1 + 1)
        nextIndex = [0] * (n1 + 1)
        j, count = 0, 0

        for k in range(1, n1 + 1):
            for i in range(len(s1)):
                if s1[i] == s2[j]:
                    j += 1
                    if j == len(s2):
                        j = 0
                        count += 1
            repeatCount[k] = count
            nextIndex[k] = j

            for start in range(k):
                if nextIndex[start] == j:
                    prefixCount = repeatCount[start]
                    patternCount = (n1 - start) // (k - start) * (repeatCount[k] - prefixCount)
                    suffixCount = repeatCount[start + (n1 - start) % (k - start)] - prefixCount
                    return (prefixCount + patternCount + suffixCount) // n2

        return repeatCount[n1] // n2